In [1]:
from tkinter import N
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer  
import pandas as pd
import wikipedia
import numpy as np
import pycountry


In [2]:
habitats_list = ['forest', 'forests', 'woods', 'woods', 'woodland', 'woodlands', 'bog', 'bogs', 'fen', 'fens', 'marsh', 'marshes', 'swamp', 'swamps', 'wetland', 'grasslands', 'prairie', 'scrubs','shrubs', 'backyards']
def get_conservation_status(df1):
    for i in range(0,len(list(df1.index))):
        if list(df1.index)[i] == 'Conservation status':
            return (list(df1.index)[i+1])
    if 'Conservation status' not in list(df1.index):
        return np.NaN
def get_kingdom(df1):
    if 'Kingdom:' in list(df1.index):
        return (df1.loc['Kingdom:'][0])
    else:
        return np.NaN
def get_phylum(df1):
    if 'Phylum:' in list(df1.index):
        return (df1.loc['Phylum:'][0])
    else:
        return np.NaN
def get_class(df1):
    if 'Class:' in list(df1.index):
        return (df1.loc['Class:'][0])
    else:
        return np.NaN
def get_order(df1):
    if 'Order:' in list(df1.index):
        return (df1.loc['Order:'][0])
    else:
        return np.NaN
def get_family(df1):
    if 'Family:' in list(df1.index):
        return (df1.loc['Family:'][0])
    else:
        return np.NaN
def get_genus(df1):
    if 'Genus:' in list(df1.index):
        return (df1.loc['Genus:'][0])
    else:
        return np.NaN
def get_species(df1):
    if 'Species:' in list(df1.index):
        return (df1.loc['Species:'][0])
    else:
        return np.NaN
def get_binomial_name(df1):
    for i in range(0,len(list(df1.index))):
        if list(df1.index)[i] == 'Binomial name':
            return(list(df1.index)[i+1])
    if 'Binomial name' not in list(df1.index):
        return np.NaN
def get_synonyms(df1):
    for i in range(0,len(list(df1.index))):
        if list(df1.index)[i] == 'Synonyms':
            return(list(df1.index)[i+1])
    if 'Synonyms' not in list(df1.index):
        return np.NaN
def get_summary(name):
    return(wikipedia.summary(name, sentences=2))
def get_categories(name):
    categories = wikipedia.page(name)
    return(categories.categories[0:3])
def get_references(name):
    refs = wikipedia.page(name)
    return(refs.references[0:3])
def get_url(name):
    url = wikipedia.page(name)
    return(url.url)
def get_image(name):
    image = wikipedia.page(name)
    return(image.images[0])
def findCountry(data):
    countries = sorted([country.name for country in pycountry.countries] , key=lambda x: -len(x))
    for country in countries:
        if country.lower() in data.lower():
            return country
    return None
def findHabitat(data):
    for habitat in habitats_list:
        if habitat in data.lower():
            return habitat
    return None
def extract_data(infobox):
    df = pd.DataFrame(infobox)
    df = df.dropna()
    row = [list(df.columns)[0]]
    
    df = df.set_index(list(df.columns)[0])
    
    row.append(get_conservation_status(df))
    row.append(get_kingdom(df))
    row.append(get_phylum(df))
    row.append(get_class(df))
    row.append(get_order(df))
    row.append(get_family(df))
    row.append(get_genus(df))
    row.append(get_species(df))
    row.append(get_binomial_name(df))
    row.append(get_synonyms(df))
    row.append(get_summary(row[0]))
    row.append(get_categories(row[0]))
    row.append(get_references(row[0]))
    row.append(get_url(row[0]))
    row.append(get_image(row[0]))
    
    birds_length = len(birds)
    birds.loc[birds_length] = row

In [3]:
startTime = datetime.now()
driver = webdriver.Chrome('./chromedriver.exe')

C:\Users\LEGION\AppData\Local\Temp\ipykernel_26032\2716060403.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [4]:
types_of_birds_df = pd.read_csv('assets/wiki_list_to_comp.csv')
list_of_bird_names = [ types_of_birds_df.loc[ _ ,'Common Name'] for _ in range(1,types_of_birds_df.shape[0]) ]
# f = 1
# t = len(list_of_bird_names)
# # f = 235
# # t = 237
# list_of_bird_names = list_of_bird_names[f:t]

In [5]:
# bird = 'Arabian scops owl'
# driver = webdriver.Chrome('./chromedriver.exe')
def get_page_of_bird(bird_name,driver):
    wiki = 'https://wikipedia.org/'
    driver.get(wiki)
    input_wiki = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='search']")))
    input_wiki.send_keys(bird_name)
    input_wiki.send_keys(Keys.ENTER)
    bird_url = driver.current_url
    l = bird_url.lower()
    if bird_url != wiki and 'search' not in l :
        return bird_url
    return None
# get_page_of_bird('ashjvcasjh ajhdfbasj',driver)


In [6]:
birds = pd.DataFrame(columns=['Name','Conservation status','Kingdom','Phylum','Class','Order','Family', 'Genus', 'Species', 'Binomial Name', 'Synonyms', 'Summary', 'Categories', 'References', 'Wiki URL', 'Image'])
pages = []

In [7]:
f = 2500
t = len(list_of_bird_names)
list_of_bird_names = list_of_bird_names[f:t]
# driver = webdriver.Chrome('./chromedriver.exe')
# f = 1
# t = len(list_of_bird_names)
ds = 'wiki_'+str(f)+'_'+str(t)

In [8]:
for i in list_of_bird_names:
    link = get_page_of_bird(i,driver)
    if link != None:
        pages.append(link)
len(pages)

1821

In [9]:
pages[-1]
# len(list_of_bird_names)

'https://en.wikipedia.org/wiki/%CA%BB%C5%8C%CA%BB%C5%AB'

In [10]:
driver.close()
pages_df = pd.DataFrame(pages)
pages_df.to_pickle('assets/rescrape_wiki/page'+ds+'.pkl')

In [12]:
failed = []
for i in pages:
    try:
        infoboxes = pd.read_html(i, attrs={"class":"infobox biota"})
        if len(infoboxes)==1:
                extract_data(infoboxes[0])
    except Exception:
        failed.append(i)

birds.head(5)

C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual e

,Name,Conservation status,Kingdom,Phylum,Class,Order,Family,Genus,Species,Binomial Name,Synonyms,Summary,Categories,References,Wiki URL,Image
0,Red-winged grey warbler,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Cisticolidae,"DrymocichlaHartlaub, 1881",D. incana,"Drymocichla incanaHartlaub, 1881",NaN,The red-winged grey warbler (Drymocichla incan...,"[All stub articles, Articles with 'species' mi...",[http://datazone.birdlife.org/species/factshee...,https://en.wikipedia.org/wiki/Red-winged_grey_...,https://upload.wikimedia.org/wikipedia/commons...
1,Red-winged lark,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Alaudidae,Mirafra,M. hypermetra,"Mirafra hypermetra(Reichenow, 1879)",Spilocorydon hypermetrus,The red-winged lark (Mirafra hypermetra) is a ...,"[Alaudidae stubs, All stub articles, Articles ...",[http://ibc.lynxeds.com/species/rufous-naped-l...,https://en.wikipedia.org/wiki/Red-winged_lark,https://upload.wikimedia.org/wikipedia/commons...
2,Red-winged laughingthrush,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Leiothrichidae,Trochalopteron,T. formosum,"Trochalopteron formosum(Verreaux, 1869)",Garrulax formosus,The red-winged laughingthrush (Trochalopteron ...,"[Articles with 'species' microformats, Article...",[http://ibc.hbw.com/ibc/phtml/especie.phtml?id...,https://en.wikipedia.org/wiki/Red-winged_laugh...,https://upload.wikimedia.org/wikipedia/commons...
3,Red-winged parrot,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Psittaciformes,Psittaculidae,Aprosmictus,A. erythropterus,"Aprosmictus erythropterus(Gmelin, 1788)",Psittacus erythropterus,The red-winged parrot (Aprosmictus erythropter...,"[All articles with unsourced statements, Apros...",[http://birdlife.org.au/bird-profile/red-winge...,https://en.wikipedia.org/wiki/Red-winged_parrot,https://upload.wikimedia.org/wikipedia/commons...
4,Red-winged prinia,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Cisticolidae,Prinia,P. erythroptera,"Prinia erythroptera(Jardine, 1849)",NaN,The red-winged prinia or the red-winged warble...,"[All stub articles, Articles with 'species' mi...","[http://www.hbw.com/node/58601, http://pubmed....",https://en.wikipedia.org/wiki/Red-winged_prinia,https://upload.wikimedia.org/wikipedia/commons...


In [13]:
# !pip install pycountry
birds['Countries'] = np.NaN
birds['Habitat'] = np.NaN

In [18]:
for i in range(1260, len(birds)):
    name = birds.Name[i]
    summary = wikipedia.summary(name)
    birds.Countries[i] = findCountry(summary)
    birds.Habitat[i] = findHabitat(summary)
birds

,Name,Conservation status,Kingdom,Phylum,Class,Order,Family,Genus,Species,Binomial Name,Summary,Categories,References,Wiki URL,Image,Countries,Habitat
0,Red-winged grey warbler,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Cisticolidae,"DrymocichlaHartlaub, 1881",D. incana,"Drymocichla incanaHartlaub, 1881",The red-winged grey warbler (Drymocichla incan...,"[All stub articles, Articles with 'species' mi...",[http://datazone.birdlife.org/species/factshee...,https://en.wikipedia.org/wiki/Red-winged_grey_...,https://upload.wikimedia.org/wikipedia/commons...,Central African Republic,NaN
1,Red-winged lark,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Alaudidae,Mirafra,M. hypermetra,"Mirafra hypermetra(Reichenow, 1879)",The red-winged lark (Mirafra hypermetra) is a ...,"[Alaudidae stubs, All stub articles, Articles ...",[http://ibc.lynxeds.com/species/rufous-naped-l...,https://en.wikipedia.org/wiki/Red-winged_lark,https://upload.wikimedia.org/wikipedia/commons...,None,NaN
2,Red-winged laughingthrush,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Leiothrichidae,Trochalopteron,T. formosum,"Trochalopteron formosum(Verreaux, 1869)",The red-winged laughingthrush (Trochalopteron ...,"[Articles with 'species' microformats, Article...",[http://ibc.hbw.com/ibc/phtml/especie.phtml?id...,https://en.wikipedia.org/wiki/Red-winged_laugh...,https://upload.wikimedia.org/wikipedia/commons...,China,forest
3,Red-winged parrot,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Psittaciformes,Psittaculidae,Aprosmictus,A. erythropterus,"Aprosmictus erythropterus(Gmelin, 1788)",The red-winged parrot (Aprosmictus erythropter...,"[All articles with unsourced statements, Apros...",[http://birdlife.org.au/bird-profile/red-winge...,https://en.wikipedia.org/wiki/Red-winged_parrot,https://upload.wikimedia.org/wikipedia/commons...,Australia,woodland
4,Red-winged prinia,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Cisticolidae,Prinia,P. erythroptera,"Prinia erythroptera(Jardine, 1849)",The red-winged prinia or the red-winged warble...,"[All stub articles, Articles with 'species' mi...","[http://www.hbw.com/node/58601, http://pubmed....",https://en.wikipedia.org/wiki/Red-winged_prinia,https://upload.wikimedia.org/wikipedia/commons...,Central African Republic,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,"(Zappey's flycatcher, Conservation status)",NaN,Animalia,Chordata,Aves,Passeriformes,Muscicapidae,Cyanoptila,C. cumatilis,"Cyanoptila cumatilis(Thayer & Bangs, 1909)",The conservation status of a group of organism...,[All articles containing potentially dated sta...,"[http://www.inbo.be/content/homepage_en.asp, h...",https://en.wikipedia.org/wiki/Conservation_status,https://upload.wikimedia.org/wikipedia/commons...,None,None
1805,"(Zenker's honeyguide, Conservation status)",NaN,Animalia,Chordata,Aves,Piciformes,Indicatoridae,Melignomon,M. zenkeri,"Melignomon zenkeriReichenow, 1898",The conservation status of a group of organism...,[All articles containing potentially dated sta...,"[http://www.inbo.be/content/homepage_en.asp, h...",https://en.wikipedia.org/wiki/Conservation_status,https://upload.wikimedia.org/wikipedia/commons...,None,None
1806,Brown-headed parrot,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Psittaciformes,Psittacidae,Poicephalus,P. cryptoxanthus,"Poicephalus cryptoxanthus(Peters, 1854)",The brown-headed parrot (Poicephalus cryptoxan...,"[All articles with unsourced statements, Artic...",[http://datazone.birdlife.org/species/factshee...,https://en.wikipedia.org/wiki/Brown-headed_parrot,https://upload.wikimedia.org/wikipedia/commons...,None,None
1807,Yellow-fronted parrot,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Psittaciformes,Psittacidae,Poicephalus,P. flavifrons,"Poicephalus flavifrons(Rüppell, 1845)",The yellow-fronted parrot (Poicephalus flavifr...,"[Articles with 'species' microformats, Article...",[http://pdfserve.info

In [19]:
ds = 'wiki_'+str(f)+'_'+str(t)
logging.basicConfig(filename="assets/rescrape_wiki/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
birds.to_pickle("assets/rescrape_wiki/Pick_"+ds+".pkl")
birds.to_csv("assets/rescrape_wiki/csv_"+ds+".csv")
if len(failed) != 0:
    fail = pd.DataFrame(failed)
    fail.to_csv("assets/rescrape_wiki/failed_"+ds+".csv")
logging.info('\n\nTime taken : ',datetime.now() - startTime)
print('\n\nTime taken : ',datetime.now() - startTime) 



Time taken :  23:37:14.235394


In [20]:
birds = birds.drop('Synonyms',axis = 1)
birds = birds.dropna(thresh=3, axis = 0)
birds.head(5)

KeyError: "['Synonyms'] not found in axis"

In [21]:
def clean_Name(df):
    sub_list = ["Conservation", "status", "Scientific", "classification"]
    for i in range(0, len(df)):
        for sub in sub_list:
            df.Name[i] = df.Name[i].replace(sub, ' ')
        df.Name[i] = df.Name[i].translate(str.maketrans('', '',string.punctuation))
        df.Name[i] = df.Name[i].strip()
        print(df.Name[i])
birds['Conservation status'] = birds['Conservation status'].astype(str)

In [22]:
def clean_Conservation_Status(df):
    for i in range(0, len(df)): 
        df['Conservation status'][i] = df['Conservation status'][i].split('(',1)[0]
        print(df['Conservation status'][i])

In [23]:
def clean_Categories_and_References(df):
    remove_list = '[]\"\''
    for i in range(0, len(df)):
        df.Categories[i] = df.Categories[i].translate(str.maketrans('', '', remove_list))
        df.References[i] = df.References[i].translate(str.maketrans('', '', remove_list))
        print(df.Categories[i])
        print(df.References[i])

In [24]:
# clean_Categories_and_References(birds)
clean_Conservation_Status(birds)
# clean_Name(birds)

file_name = 'assets/rescrape_wiki/bird_'+ds+'_dataset.xlsx'

Least Concern 
Least Concern 
Least Concern 
Least Concern 
Least Concern 
nan
Least Concern 
Least Concern 
Least Concern 
Least Concern 
Near Threatened 
Vulnerable 
Least Concern 
Least Concern 
Critically Endangered 
Least Concern 
Least Concern 
nan
Least Concern 
nan
Least Concern 
Range Restricted 
Least Concern 
Vulnerable 
nan
nan
Near Threatened 
nan
nan
nan
Near Threatened 
Least Concern 
Least Concern 
Least Concern 
Least Concern 
Vulnerable 
Near Threatened 
Least Concern 
nan
Critically Endangered 
Near Threatened 
Least Concern 
Least Concern 
Least Concern 
Least Concern 
Least Concern 
nan
Least Concern 
Least Concern 
Least Concern 
Least Concern 
Near Threatened 
Vulnerable 
Least Concern 
Least Concern 
Critically Endangered 
Least Concern 
Least Concern 
nan
Least Concern 
nan
Least Concern 
Range Restricted 
Least Concern 
Vulnerable 
nan
nan
Near Threatened 
nan
nan
nan
Near Threatened 
Least Concern 
Least Concern 
Least Concern 
Least Concern 
Vulnerable 
Near

In [25]:
birds.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
